In [6]:
from datasets import load_dataset, load_metric, DatasetDict

In [17]:
pos_tag_thai = load_dataset("json", data_files="th_pud_data.json")
train_test_pos_tag = pos_tag_thai['train'].train_test_split(test_size=0.2)
valid_pos = train_test_pos_tag['test'].train_test_split(test_size=0.5)

pos_tag_thai = DatasetDict({
    'train': train_test_pos_tag['train'],
    'test': valid_pos['train'],
    'valid': valid_pos['test']
})

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 163.61it/s]
Generating train split: 1 examples [00:00,  2.76 examples/s]


ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [8]:
pos_tag_thai

DatasetDict({
    train: Dataset({
        features: ['id', 'pos_tag', 'token'],
        num_rows: 104448
    })
    test: Dataset({
        features: ['id', 'pos_tag', 'token'],
        num_rows: 13056
    })
    valid: Dataset({
        features: ['id', 'pos_tag', 'token'],
        num_rows: 13056
    })
})

In [16]:
pos_tag_thai['train'][0]['pos_tag']

'['